# 01_09_remove_Bad_fits

## 필요한 모듈

이 프로젝트를 위해서는 아래의 모듈이 필요하다. 

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, version_information

### 모듈 설치

1. 콘솔 창에서 모듈을 설치할 때는 아래와 같은 형식으로 입력하면 된다.

>pip install module_name==version

>conda install module_name=version

2. 주피터 노트북(코랩 포함)에 설치 할 때는 아래의 셀을 실행해서 실행되지 않은 모듈을 설치할 수 있다. (pip 기준) 만약 아나콘다 환경을 사용한다면 7행을 콘다 설치 명령어에 맞게 수정하면 된다.

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [181]:
# import importlib, sys, subprocess
# packages = "numpy, matplotlib, astropy, astroquery, version_information" # required modules
# pkgs = packages.split(", ")
# for pkg in pkgs :
#     if not importlib.util.find_spec(pkg):
#         #print(f"**** module {pkg} is not installed")
#         subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
#     else: 
#         print(f"**** module {pkg} is installed")

# %load_ext version_information
# import time
# now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
# print(f"This notebook was generated at {now} ")

# vv = %version_information {packages}
# for i, pkg in enumerate(vv.packages):
#     print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

### import modules

In [182]:
from glob import glob
from pathlib import Path
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.stats import sigma_clip
from ccdproc import combine, ccd_process, CCDData
from pathlib import Path

import ysfitsutilpy as yfu
import ysphotutilpy as ypu
from astropy.io import fits
import _astro_utilities
import _Python_utilities

plt.rcParams.update({'figure.max_open_warning': 0})

In [183]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/OBS_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-03_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-05_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2017-06_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2021-10_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2022-01_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C2-Asteroid"
TODODIR = PROJECDIR / "-_-_-_2022-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2022-_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2023-_-_RiLA600_STX-16803_-_2bin"

PROJECDIR = BASEDIR / "C3-EXO"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-05_-_RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "-_-_-_2024-06_-_RiLA600_STX-16803_-_2bin"
TODODIR = PROJECDIR / "-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin"

# PROJECDIR = BASEDIR / "C4-Spectra"
# TODODIR = PROJECDIR / "-_-_-_2024-05_TEC140_ASI183MMPro_-_1bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])
    
except : 
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'Kepler-17b_LIGHT_-_2024-06-26_-_RiLA600_STX-16803_-_2bin'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))

DOINGDIRs:  ['/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024-09_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-6b_LIGHT_-_2024-09-24_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-93b_LIGHT_-_2024-09-26_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/KELT-1b_LIGHT_-_2024-09-23_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/TOI-1518b_LIGHT_-_2024-09-10_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/WASP-33b_LIGHT_-_2024-09-28_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/WASP-52b_LIGHT_-_2024-09-27_-_GSON3

In [184]:
DOINGDIR = Path(DOINGDIRs[0])

In [185]:
DOINGDIR.parent / DOINGDIR.stem
fpaths = sorted(list((DOINGDIR / "Bad_fits").glob('*.fit*')))
print(f"fpaths: {fpaths}")
# for fpath in fpaths :
#     if fpath

fpaths: [PosixPath('/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin/Bad_fits/HAT-P-59b_LIGHT_R_2024-09-07-13-41-09_150sec_GSON300_STF-8300M_-10c_1bin.fit'), PosixPath('/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin/Bad_fits/HAT-P-59b_LIGHT_R_2024-09-07-13-46-53_150sec_GSON300_STF-8300M_-10c_1bin.fit'), PosixPath('/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin/Bad_fits/HAT-P-59b_LIGHT_R_2024-09-07-13-52-15_150sec_GSON300_STF-8300M_-10c_1bin.fit'), PosixPath('/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin/Bad_fits/HAT-P-59b_LIGHT_R_2024-09-07-13-57-37_150sec_GSON300_STF-8300M_-10c_1bin.fit'), PosixPath('/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_

In [186]:
for DOINGDIR in DOINGDIRs[:] :
    DOINGDIR = Path(DOINGDIR)
    print(DOINGDIR)
    # fpath = Path(fpaths[0])
    fpaths = sorted(list((DOINGDIR / "Bad_fits").glob('*.fit*')))
    for fpath in fpaths :
        fpath = Path(fpath)
        # print(fpath)
        if (fpath.parents[1] / _astro_utilities.reduced_dir / fpath.name).exists() :
            os.remove (fpath.parents[1] / _astro_utilities.reduced_dir / fpath.name)
            print (f"{fpath.parents[1] / _astro_utilities.reduced_dir / fpath.name} is removed...")
        if (fpath.parents[1] / _astro_utilities.reduced_nightsky_dir / fpath.name).exists() :
            os.remove (fpath.parents[1] / _astro_utilities.reduced_nightsky_dir / fpath.name)
            print (f"{fpath.parents[1] / _astro_utilities.reduced_nightsky_dir / fpath.name} is removed...")

/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-59b_LIGHT_-_2024-09-07_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-6b_LIGHT_-_2024-09-24_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/HAT-P-93b_LIGHT_-_2024-09-26_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/KELT-1b_LIGHT_-_2024-09-23_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/TOI-1518b_LIGHT_-_2024-09-10_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/WASP-33b_LIGHT_-_2024-09-28_-_GSON300_STF-8300M_-_1bin
/mnt/Rdata/OBS_data/C3-EXO/-_-_-_2024-09_-_GSON300_STF-8300M_-_1bin/WASP-52b_LIGHT_-_2024-09-27_-_GSON300_STF-8300M_-_1bin


In [187]:
# DOINGDIR = Path(DOINGDIRs[0])
# print("DOINGDIR", DOINGDIR)

# shutil.move(f"{DOINGDIR/"REDUC_nightsky"}", f"{DOINGDIR/_astro_utilities.reduced_nightsky_dir}")
# print(f"{DOINGDIR/"REDUC_nightsky"} is move to {DOINGDIR/_astro_utilities.reduced_nightsky_dir} ...")
# # shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# # print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")


In [188]:
# for DOINGDIR in DOINGDIRs[:] :
#     DOINGDIR = Path(DOINGDIR)
#     print("DOINGDIR", DOINGDIR)
#     # print((DOINGDIR / "REDUC_nightsky"))
#     # print((DOINGDIR / "REDUC_nightsky").exists())
#     if (DOINGDIR/"reduced_DPhot_Mag12.5").exists() and not (DOINGDIR/"reduced_DPhot_Mag12.5_fw6").exists():
#         shutil.move(DOINGDIR/"reduced_DPhot_Mag12.5",
#                      DOINGDIR/"reduced_DPhot_Mag12.5_fw6")
#     if (DOINGDIR/"reduced_DPhot_Mag11").exists() and not (DOINGDIR/"reduced_DPhot_Mag11_fw6").exists():
#         shutil.move(DOINGDIR/"reduced_DPhot_Mag11",
#                      DOINGDIR/"reduced_DPhot_Mag11_fw6")
#     if (DOINGDIR/"reduced_nightsky_DPhot_Mag12.5").exists() and not (DOINGDIR/"reduced_nightsky_DPhot_Mag12.5_fw6").exists():
#         shutil.move(DOINGDIR/"reduced_nightsky_DPhot_Mag12.5",
#                      DOINGDIR/"reduced_nightsky_DPhot_Mag12.5_fw6")
#     if (DOINGDIR/"reduced_nightsky_DPhot_Mag11").exists() and not (DOINGDIR/"reduced_nightsky_DPhot_Mag11_fw6").exists():
#         shutil.move(DOINGDIR/"reduced_nightsky_DPhot_Mag11",
#                      DOINGDIR/"reduced_nightsky_DPhot_Mag11_fw6")



In [189]:
# for DOINGDIR in DOINGDIRs[:] :
#     DOINGDIR = Path(DOINGDIR)
#     print("DOINGDIR", DOINGDIR)
#     removing_lst = [
#                     # _astro_utilities.master_dir,
#                     # _astro_utilities.reduced_dir,
#                     # _astro_utilities.reduced_nightsky_dir,
#                     # _astro_utilities.Asteroid_result_dir,
#                     # _astro_utilities.Diff_Phot_dir,
#                     "reduced_DPhot_Mag12.5",
#                     "reduced_nightsky_DPhot_Mag12.5",
#                     ]
#     for dir_name in removing_lst :
#         if (DOINGDIR/dir_name).exists() :
#             print(f"{DOINGDIR/dir_name} is removed...")
#             shutil.rmtree(str(DOINGDIR/dir_name), ignore_errors=True)

In [190]:
# import shutil
# DOINGDIR = Path(DOINGDIRs[0])
# print("DOINGDIR", DOINGDIR)

# # shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# # print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")
# shutil.rmtree(DOINGDIR/_astro_utilities.master_dir)
# print(f"{DOINGDIR/_astro_utilities.master_dir} is removed...")
